## Searching for bouts for a day of alsa recording and the pyrec generated wav, bout dicts

In [81]:
%matplotlib inline

import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2023-02-28 14:31:08,591 root         INFO     Running on txori.ucsd.edu
2023-02-28 14:31:08,591 root         INFO     Running on txori.ucsd.edu


In [82]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.pipeline import searchbout as sb

### Get the file locations for a session (day) of recordings

In [88]:
reload(et)
sess_par = {'bird': 's_b1496_23',
           'sess': '2023-02-08',
            'acq_software': 'pyrecord',
           'derived_folder': 'bouts_ceciestunepipe',
           'auto_file': 'bout_auto_file',
           'curated_file': 'bout_curated_file',
           'super_session': 'all-sess-01'}

#pyrecord is where raw data is coming from the script that detects the song 
exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software=sess_par['acq_software'])

raw_folder = exp_struct['folders'][sess_par['acq_software']]
bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')

In [89]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1496_23',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08',
 'pyrecord': '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord',
 'kwik': '/scratch/earneodo/s_b1496_23/pyrecord/kwik/2023-02-08',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1496_23/2023-02-08/pyrecord',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1496_23/2023-02-08/pyrecord',
 'tmp': '/scratch/earneodo/tmp',
 'msort': '/scratch/earneodo/s_b1496_23/pyrecord/msort/2023-02-08',
 'ksort': '/scratch/earneodo/s_b1496_23/pyrecord/ksort/2023-02-08/',
 'sort': '/mnt/sphere/speech_bci/derived_data/s_b1496_23/2023-02-08/pyrecord/'}

### read a session's bouts files
- get the location of the file
- load the pandas pickle
- load the search/spectrogram parameters

In [90]:
#raw_folder = exp_struct['folders'][sess_par['acq_software']]
#!ls $raw_folder

In [91]:
import json

In [92]:
glob.glob(os.path.join(raw_folder, '*.json'))[:5]
#each time it triggers it makes two files, the microphone recording and a json file with the metadata 
#rec_offset_ms is the buffer of the recording
#rec_time_stamp is when the recording started (something longer than 7s is gonna get marked as a bout)
#whenever we say it is a bout, what does the ground truth say that is happening --> your job slackers
#want to make an estimate of how many we have in one that are not there in the other 
#we want a plot where each timestamp is a dot/two dots/no dot? one for the ground truth and one for the other one
#we want to look for where the ground truth matches this thing 

['/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230208_152933_sound.json',
 '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230209_085719_sound.json',
 '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230209_103121_sound.json',
 '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230209_140017_sound.json',
 '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230208_191701_sound.json']

In [93]:
def load_json_bout(bout_path):
    with open(bout_path, 'r') as fp:
        bout_dict = json.load(fp)
        bout_dict['file_path'] = bout_path
    return bout_dict

def load_all_json_bouts(raw_folder: str):
    bout_path_list = glob.glob(os.path.join(raw_folder, '*_sound.json'))
    bout_dict_list = [load_json_bout(d) for d in bout_path_list]
    return bout_dict_list

dict_list = load_all_json_bouts(raw_folder)

In [94]:
dict_list[:5]

[{'wav_file_name': '/mnt/data/raw_data/s_b1496_22/2023-02-08/pyrecord/20230208_152933_sound.wav',
  'json_file_name': '/mnt/data/raw_data/s_b1496_22/2023-02-08/pyrecord/20230208_152933_sound.json',
  'rec_offset_ms': 3000,
  'rec_time_stamp': '20230208_152930.63',
  'bout_time_stamp': '20230208_152937.72',
  'bout_offset_ms': None,
  'is_bout': True,
  'bout_ms_in_buf': 10005,
  'len_ms': 36580,
  'file_path': '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230208_152933_sound.json'},
 {'wav_file_name': '/mnt/data/raw_data/s_b1496_22/2023-02-08/pyrecord/20230209_085719_sound.wav',
  'json_file_name': '/mnt/data/raw_data/s_b1496_22/2023-02-08/pyrecord/20230209_085719_sound.json',
  'rec_offset_ms': 3000,
  'rec_time_stamp': '20230209_085716.57',
  'bout_time_stamp': None,
  'bout_offset_ms': None,
  'is_bout': False,
  'file_path': '/mnt/sphere/speech_bci/raw_data/s_b1496_23/2023-02-08/pyrecord/20230209_085719_sound.json'},
 {'wav_file_name': '/mnt/data/raw_data/s_b149

In [118]:
# Get all the timestamps that have a label is_bout: True for all the recordings of the raspi
# For the ones where is_bout = False there's a None for the bout_time_stamp 
key = 'is_bout' #to check if it's a bout
keyis = 'bout_time_stamp' #to get the timestamp of the bout
timestamps_list = []
for i in range(len(dict_list)): #iterate through list
    if dict_list[i][key] == True: #check if label is_bout is True
        timestamps_list.append(dict_list[i][keyis]) #if it is then append that timestamp
# res = list(filter(lambda item: item is not None, timestamps_list)) #remove all Nones from list
len(timestamps_list)
timestamps_list

['20230208_152937.72',
 '20230209_140021.91',
 '20230208_191705.39',
 '20230208_194421.90',
 '20230209_063711.78',
 '20230209_071308.77',
 '20230208_192042.32',
 '20230208_204443.95',
 '20230209_104739.62',
 '20230208_165514.49',
 '20230209_072231.73',
 '20230208_202955.20',
 '20230209_133613.85',
 '20230209_060645.97',
 '20230209_102504.33',
 '20230209_062330.52',
 '20230209_083553.21',
 '20230209_075606.93',
 '20230208_164304.94',
 '20230209_114638.88',
 '20230209_062748.28',
 '20230209_122513.65',
 '20230209_075642.64',
 '20230208_193616.37',
 '20230208_160916.77',
 '20230209_093405.77',
 '20230208_202649.07',
 '20230209_060443.24',
 '20230209_103534.72',
 '20230209_090342.06',
 '20230209_125740.85',
 '20230209_143333.11',
 '20230209_115608.04',
 '20230209_091107.79',
 '20230209_083735.15',
 '20230209_133406.15',
 '20230209_091941.47',
 '20230209_083102.90',
 '20230208_194836.18',
 '20230209_094307.14',
 '20230209_115343.96',
 '20230209_085416.53',
 '20230209_144938.11',
 '20230209_